# Статистика по продуктам

In [2]:
import matplotlib.pyplot as plt
import psycopg2 as psy
import pandas as pd

from datetime import datetime as dt

def state_flag(row):
    if row['shipped_date'] == None:
        return 1
    else:
        return 0

def sql_to_pd(query, insert_state_flag=False):
    '''
    Функция стучится в базу данных и обрабатывает указанный в аргументе запрос.
    Затем преобразует полученные данные в датафрейм и возвращает его.

    query - SQL запрос в базу данных, которые указанны в функции.
    
    insert_state_flag - подходит исключительно для БД из задания.
                        Если активен, добавляет столбец со статусом
                        заказа в датафрейм используя функцию state_flag()
                        (0-неактивный, 1-активный)
    '''
    try:
        # Connect to an existing database
        connection = psy.connect(user="postgres",
                                  password="Pga1708844!",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres")
        # Create a cursor to perform database operations
        cursor = connection.cursor()
        cursor.execute(query)
        colnames = [desc[0] for desc in cursor.description]
        record = cursor.fetchall()
        # Create a dataframe with the data from a database
        data_frame = pd.DataFrame(record, columns=colnames)
        # Использую функцию выше в случае если нужно поставить флаг статуса заказа, включен по умолчанию
        if insert_state_flag:
            data_frame['order_state'] = data_frame.apply(state_flag, axis=1)
        return data_frame
    except Exception as e:
        print("Error while connecting to PostgreSQL", e)
    finally:
        if (connection):
            cursor.close()
            connection.close()

## 1. Выведите клиента с наибольшим количеством активных заказовМетод to_excel)

In [3]:
df = sql_to_pd('SELECT * FROM orders\
                    INNER JOIN order_details USING(order_id)\
                    INNER JOIN customers USING(customer_id)'
               , insert_state_flag=True
              )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       2155 non-null   object 
 1   order_id          2155 non-null   int64  
 2   employee_id       2155 non-null   int64  
 3   order_date        2155 non-null   object 
 4   required_date     2155 non-null   object 
 5   shipped_date      2082 non-null   object 
 6   ship_via          2155 non-null   int64  
 7   freight           2155 non-null   float64
 8   ship_name         2155 non-null   object 
 9   ship_address      2155 non-null   object 
 10  ship_city         2155 non-null   object 
 11  ship_region       856 non-null    object 
 12  ship_postal_code  2100 non-null   object 
 13  ship_country      2155 non-null   object 
 14  product_id        2155 non-null   int64  
 15  unit_price        2155 non-null   float64
 16  quantity          2155 non-null   int64  


In [4]:
highest_active = df\
                .groupby('customer_id')\
                .agg({'company_name': 'first', 'contact_name': 'first', 'address': 'first', 'order_state':'sum'})\
                .sort_values(by='order_state', ascending=False)\
                .reset_index()
highest_active.rename(columns={'order_state': 'active_orders_count'}, inplace=True)
highest_active.head(1)

,customer_id,company_name,contact_name,address,active_orders_count
0,RATTC,Rattlesnake Canyon Grocery,Paula Wilson,2817 Milton Dr.,25


## 2. Выведите клиентов по количеству активных и неактивных заказов

In [26]:
non_active_orders = df[df['order_state'] == 0][['customer_id','company_name', 'contact_name', 'address', 'order_state']]
non_active_orders = non_active_orders\
                    .groupby('customer_id')\
                    .agg({'company_name': 'first', 'contact_name': 'first', 'address': 'first', 'order_state':'count'})
non_active_orders.rename(columns={'order_state': 'non_active_count'}, inplace=True)

active_orders = df[df['order_state'] > 0][['customer_id','company_name', 'contact_name', 'address', 'order_state']]
active_orders = active_orders\
                    .groupby('customer_id')\
                    .agg({'company_name': 'first', 'contact_name': 'first', 'address': 'first', 'order_state':'count'})
active_orders.rename(columns={'order_state': 'active_count'}, inplace=True)

non_active_orders.merge(active_orders, how='left').fillna(0)

# список всех клиентов и количесво актиных и неактивных заказов

,company_name,contact_name,address,non_active_count,active_count
0,Alfreds Futterkiste,Maria Anders,Obere Str. 57,12,0.0
1,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,10,0.0
2,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,17,0.0
3,Around the Horn,Thomas Hardy,120 Hanover Sq.,30,0.0
4,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,52,0.0
...,...,...,...,...,...
84,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,37,0.0
85,Wellington Importadora,Paula Parente,"Rua do Mercado, 12",19,0.0
86,White Clover Markets,Karl Jablonski,305 - 14th Ave. S. Suite 3B,40,0.0
87,Wilman Kala,Matti Karttunen,Keskuskatu 45,17,0.0


In [35]:
# однако будет более информативным ваиратнт где мы выберем только тех клиентов которые имеют как активные так и неактивные заказы
clients = non_active_orders.merge(active_orders, how='left').fillna(0)
clients = clients[(clients['active_count'] != 0) & (clients['non_active_count'] != 0)]
clients['active_count'] = clients['active_count'].astype(int)
clients = clients.sort_values(by=['active_count', 'non_active_count'], ascending=False)\
                 .reset_index() # сортирую результат по активным заказам
clients

,index,company_name,contact_name,address,non_active_count,active_count
0,62,Rattlesnake Canyon Grocery,Paula Wilson,2817 Milton Dr.,46,25
1,19,Ernst Handel,Roland Mendel,Kirchgasse 6,95,7
2,42,Lehmanns Marktstand,Renate Messner,Magazinweg 7,35,4
3,45,LINO-Delicateses,Felipe Izquierdo,Ave. 5 de Mayo Porlamar,31,4
4,44,LILA-Supermercado,Carlos González,Carrera 52 con Ave. Bolívar #65-98 Llano Largo,30,4
5,8,Bon app',Laurence Lebihan,"12, rue des Bouchers",41,3
6,59,Queen Cozinha,Lúcia Carvalho,"Alameda dos Canàrios, 891",37,3
7,65,Richter Supermarkt,Michael Holz,Grenzacherweg 237,27,3
8,64,Ricardo Adocicados,Janete Limeira,"Av. Copacabana, 267",24,3
9,5,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,11,3


## 3. Нарисуйте круговую диаграмму по топ 10 самых больших по заказам клиентов, разделенных на активные и неактивные заказы

In [ ]:
import plotly.express as px

px.

## 4. Выведите продукт с наибольшим количеством активных зказов

## 5. Выведите количества продуктов разделенные по активности

## 6. Нарисуйте диаграмму по топ 10 самым продаваемым продуктам

## 7. Добавьте в вашу таблицу должность и имя сотрудника, а также его город, гдеон работает

## 8. Теперь выведите количество активных и неактивных заказов по сотрудникам

## 9. Выгрузите таблицу с заказами и сотрудниками в эксель. (Метод to_excel)